1. 找出每个句子的主题
2. 相同主题出现的次数
3. 整个主题的可视化

In [75]:
#挂载云盘 
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [76]:
#进入项目目录
!ls
import os
os.chdir('/content/drive/My Drive/Project/LDA项目')
!ls

 LDA.ipynb	    record_topic.gsheet   内容.xlsx
 record_topic.csv  'stopwords (1).txt'
 LDA.ipynb	    record_topic.gsheet   内容.xlsx
 record_topic.csv  'stopwords (1).txt'


In [79]:
!pip install gensim
import numpy as np
import pandas as pd
import re
from gensim import corpora, models, similarities
import gensim
#参数设置
num_topics = 10#主题数量
topn = 3#主题向量前几个

# 用正则表达式清洗数据
def clean_email_text(text):
    text = text.replace('\n'," ")                        # 新行，我们是不需要的
    text = re.sub(r"-", " ", text)                       # 把 "-" 的两个单词，分开。（比如：july-edu ==> july edu）
    text = re.sub(r"\d+/\d+/\d+", "", text)              # 日期，对主体模型没什么意义
    text = re.sub(r"[0-2]?[0-9]:[0-6][0-9]", "", text)   # 时间，没意义
    text = re.sub(r"[\w]+@[\.\w]+", "", text)            # 邮件地址，没意义
    text = re.sub(r"/[a-zA-Z]*[:\//\]*[A-Za-z0-9\-_]+\.+[A-Za-z0-9\.\/%&=\?\-_]+/i", "", text)    # 网址，没意义    
    # 以防还有其他除了单词以外的特殊字符（数字）等等，我们把特殊字符过滤掉
    # 只留下字母和空格
    # 再把单个字母去掉，留下单词
    pure_text = ''
    for letter in text:
        if letter.isalpha() or letter==' ':
            pure_text += letter
            
    text = ' '.join(word for word in pure_text.split() if len(word)>1)
    return text

"""第一步：用正则表达式清洗数据，并去除停用词"""
print("展示下表格数据: \n")
data = pd.read_excel('内容.xlsx',header=None)
display(data.head(2))
#数据清洗
docs_text = data[0]
docs = docs_text.apply(lambda s: clean_email_text(s))  
# 得到所有文本的内容
doclist = docs.values
print("一共有",len(doclist),"段文本。\n")
print("第1段文本未清洗前的内容为: \n",docs_text.iloc[0],'\n')
# 去除停用词，处理成gensim需要的输入格式
stopwords = [word.strip() for word in open('stopwords (1).txt','r').readlines()]
# 每一段文本都有星期和月份，这里也把他们过滤掉
weeks = ['monday','mon','tuesday','tues','wednesday','wed','thursday','thur','friday','fri','saturday','sat','sunday','sun']
months = ['jan','january','feb','february','mar','march','apr','april','may','jun','june','jul',\
          'july','aug','august','sept','september','oct','october','nov','november','dec','december']
others = ['th']#,'covid','coronavirus'
stoplist = stopwords+weeks+months+others+['am','pm']
texts = [[word for word in doc.lower().split() if word not in stoplist] for doc in doclist]
print("第1段文本去除停用词并处理成gensim需要的格式为：\n",texts[0],'\n')

"""第二步：构建字典，将文本ID化"""
dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]
# 将每一篇文本ID化
print("第1段文本ID化后的结果为：\n",corpus[0],'\n')

"""第三步：训练LDA模型"""
lda = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=dictionary, num_topics=num_topics)
print("LDA模型训练完成！\n")

#查看十个主题分别是什么单词
print("查看%d个主题分别是什么单词：\n"%num_topics)
for i in range(num_topics):
  print(lda.print_topic(i, topn=topn))

#记录每段文本的主题
record_topic=[]
for i in range(len(doclist)):
  topic = lda.get_document_topics(corpus[i])
  record_topic.append(topic[0][0])
pd.DataFrame(record_topic).to_csv('record_topic.csv')
print("已记录每段文本的主题到record_topic.csv！\n")

#统计每个主题出现的次数
times_dic={}
for i in range(num_topics):
  times_dic[i] = 0
for i in record_topic:
  times_dic[i] += 1
print("已统计每个主题出现的次数！\n")
print(times_dic)

展示下表格数据: 



,0
0,CRISPR pioneer Doudna opens lab to run Covid-1...
1,Coronavirus Doctors Threatened to Be Fired if ...


一共有 1291 段文本。

第1段文本未清洗前的内容为: 
 CRISPR pioneer Doudna opens lab to run Covid-19 tests. #21stcenturytech #COVID19  https://statnews.com/2020/03/30/crispr-pioneer-doudna-opens-lab-to-run-covid-19-tests/… via @statnews 

第1段文本去除停用词并处理成gensim需要的格式为：
 ['crispr', 'pioneer', 'doudna', 'opens', 'lab', 'run', 'covid', 'tests', 'stcenturytech', 'covid', 'httpsstatnewscomcrispr', 'pioneer', 'doudna', 'opens', 'lab', 'run', 'covid', 'tests', 'via', 'statnews'] 

第1段文本ID化后的结果为：
 [(0, 3), (1, 1), (2, 2), (3, 1), (4, 2), (5, 2), (6, 2), (7, 2), (8, 1), (9, 1), (10, 2), (11, 1)] 

LDA模型训练完成！

查看10个主题分别是什么单词：

0.036*"covid" + 0.009*"help" + 0.008*"coronavirus"
0.063*"covid" + 0.021*"coronavirus" + 0.008*"people"
0.042*"covid" + 0.021*"coronavirus" + 0.008*"new"
0.040*"covid" + 0.014*"coronavirus" + 0.008*"health"
0.054*"covid" + 0.009*"coronavirus" + 0.007*"us"
0.051*"covid" + 0.006*"coronavirus" + 0.006*"pandemic"
0.047*"covid" + 0.024*"coronavirus" + 0.008*"cases"
0.052*"covid" + 0.023*"coronavirus" 